In [21]:
#import dependencies
import pymongo
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import requests
import time

In [22]:
#enable chrome browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [23]:
#assign URLs
nasa_url= 'https://mars.nasa.gov/news/'
jpl_image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
nasa_twitter_url = 'https://twitter.com/marswxreport?lang=en'
mars_facts_url = 'https://space-facts.com/mars/'
usgs_image_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

## Retrieve NASA MARS News

In [24]:
#retrieve the page using requests
#nasa_response = requests.get(nasa_url)
browser.visit(nasa_url)
time.sleep(3)
nasa_response = browser.html

#create a soup variable for nasa url
nasa_soup = bs(nasa_response, 'lxml')

In [25]:
nasa_results = nasa_soup.find('li', class_='slide')

In [26]:
for x in nasa_results:
    news_body = x.find('div', class_='rollover_description_inner').text
    news_title = x.find('div', class_= 'content_title').text

print(f"Article Title: \n{news_title} \n\nArticle_Description: \n{news_body}")

Article Title: 
NASA's Perseverance Rover Will Look at Mars Through These 'Eyes' 

Article_Description: 
A pair of zoomable cameras will help scientists and rover drivers with high-resolution color images.


## Retrieve JPL Image

In [27]:
base_url = 'https://jpl.nasa.gov'
browser.visit(jpl_image_url)
jpl_response = browser.html

#create a soup variable for nasa url
jpl_soup = bs(jpl_response, 'lxml')

In [28]:
browser.click_link_by_id('full_image')

In [29]:
browser.click_link_by_partial_text('more info')

In [30]:
#reset html and soup
jpl_response = browser.html
jpl_soup = bs(jpl_response, 'lxml')

In [31]:
images = jpl_soup.find_all('img', class_='main_image')

for image in images:
    jpl_image = base_url + image['src']
    
browser.visit(jpl_image)    
print(jpl_image)

https://jpl.nasa.gov/spaceimages/images/largesize/PIA16029_hires.jpg


## Retrieve MARS Weather

In [32]:
#use requests to navigate to nasa twitter url
response = requests.get(nasa_twitter_url)
twitter_soup = bs(response.text, 'lxml')

#create list to store tweets
tweets = []

In [33]:
#loop through content and store tweets in tweets list
for tweet in twitter_soup.find_all('div', class_='content'):
    tweets.append(tweet.find('div',class_='js-tweet-text-container').find('p').text)

#assign 1st tweet to mars weather variable
    mars_weather = tweets[0]
print(mars_weather)

On this day in 1982, the @weatherchannel  went on the air.pic.twitter.com/nP22EkDLVw


## Retrieve MARS Facts

In [34]:
#use pandas read_html to pull the tables from this webpage
facts_tables = pd.read_html(mars_facts_url)

#set 1st index as dataframe
facts_df = facts_tables[0]

#set 1st column as df index
facts_df.set_index([0])

#convert and export data frame to HTML file
html = facts_df.to_html(index = True, header = False)

facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


## Mars Hemispheres

In [35]:
#nasa_response = requests.get(nasa_url)
base_url = 'https://astrogeology.usgs.gov/'
browser.visit(usgs_image_url)
usgs_response = browser.html

#create a soup variable for nasa url
usgs_soup = bs(usgs_response, 'lxml')

In [40]:
#create lists to store data
title = []
url = []
img_url = []

#pull title names and append to title list
for item in usgs_soup.find_all('div', class_='item'):
    title.append(item.find('h3').text)    

#print titles
for x in title:
    print(x)

Cerberus Hemisphere Enhanced
Schiaparelli Hemisphere Enhanced
Syrtis Major Hemisphere Enhanced
Valles Marineris Hemisphere Enhanced


In [41]:
#find all item links
links = usgs_soup.find_all('a', class_='itemLink product-item', href=True)

#grab url and append to url list
for link in links:
    if link.text:
        x = base_url + link['href']
        url.append(x)
        print(x)    

https://astrogeology.usgs.gov//search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov//search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov//search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov//search/map/Mars/Viking/valles_marineris_enhanced


In [42]:
#loop through each url to grab image link
for page in url:
    browser.visit(page)
    url_response = browser.html
    url_soup = bs(url_response, 'lxml')
    
    #find all divs with downloads class
    divs = url_soup.find_all('div', class_= 'downloads')
    
    #loop through divs to find img urls
    for div in divs:
        image = div.find('a')['href']
        img_url.append(image)
        print(image)


http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


In [64]:
#create final dictionary and list to hold dictionary
mars_dict = {}
hemisphere_image_urls = []

#loop through title and img_url lists and append to dictionary
for x in range(len(title)):
    mars_dict = {}
    mars_dict['title'] = title[x]
    mars_dict['img_url'] = img_url[x]
    hemisphere_image_urls.append(mars_dict)

In [65]:
#preview final list of dictionary
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]